In [1]:
from tkinter import *
from gtts import gTTS
import speech_recognition as sr
import pyttsx3
from PIL import ImageTk, Image
import time
import os
import playsound
import webbrowser
import requests
import json
import wolframalpha


In [2]:
def rec_audio():
    rcg = sr.Recognizer()
    
    with sr.Microphone() as source:
        audio = rcg.listen(source, 10, 60)
    
    data = " "
    speak = " "
    try:
        data = rcg.recognize_google(audio)
        if "who are you" in data or "define yourself" in data:
            speak += """Hello, I am MicroPh, your Assistant. You can command me to perform various tasks."""
            label.configure(text=speak)
        elif "your name" in data:
            speak += "My name is MicroPh."
            label.configure(text=speak)
        elif "why do you exist" in data or "why are you here" in data:
            speak += "It is a secret."
            label.configure(text=speak)
        elif "youtube" in data.lower():
            ind = data.lower().split().index("youtube")
            search = data.split()[ind+1:]
            webbrowser.open(
                "http://www.youtube.com/results?search_query=" + "+".join(search)
            )
            speak += "Opening" + str(search) + " on youtube"
            label.configure(text=speak)
        elif "google" in data.lower() and "search" in data.lower():
            ind = data.lower().split().index("search")
            search = data.split()[ind+1:]
            webbrowser.open(
                "https://www.google.com/search?q=" + "+".join(search)
            )
            speak += "Searching" + str(search) + " on google"
            label.configure(text=speak)
        elif "where is" in data:
            ind = data.lower().split().index("is")
            location = data.split()[ind+1:]
            url = "https://www.google.com/maps/place/" + "".join(location)
            speak += "This is where " + str(location) + " is. "
            webbrowser.open(url)
            label.configure(text=speak)
        elif "joke" in data:
            if "chicken" in data:
                speak += "Why didn't the chicken cross the road? It was a comidehen."
                label.configure(text=speak)
            elif "atom" in data:
                speak += "What is smaller than an atom? A quantum bot."
                label.configure(text=speak)
            elif "women" in data:
                speak += "Why do women like men? They like the way they burp."
                label.configure(text=speak)
        elif "weather" in data:
            key = "3a3b62fd5ae2d814ec36e2ced72b56b4"
            weather_url = "http://api.openweathermap.org/data/2.5/weather?"
            ind = data.split().index("in")
            location = data.split()[ind+1:]
            location = "".join(location)
            url = weather_url + "appid=" + key + "&q=" + location
            js = requests.get(url).json()
            if js["cod"] != 404:
                weather = js["main"]
                temperature = (weather["temp"] - 273.15) * 1.8 + 32
                humidity = weather["humidity"]
                desc = js["weather"][0]["description"]
                feels_like_temp = (weather["feels_like"] - 273.15) * 1.8 + 32
                weather_response = "The temperature is " + str(round(temperature, 2)) + ' ° Fahrenheit.' + " The temperature feels like " + str(round(feels_like_temp, 2)) + ' ° Fahrenheit.' + " The humidity is " + str(humidity) + ' percent.' + " Finally, the weather can be described as " + str(desc) + '.' 
                speak += weather_response
                label.configure(text=speak)
            else:
                speak += "City not found"
                label.configure(text=speak)
        elif "calculate" in data:
            app_id = "75H7YR-8T56GVTREH"
            client = wolframalpha.Client(app_id)
            ind = data.lower().split().index("calculate")
            data = data.split()[ind+1:]
            res = client.query(" ".join(data))
            answer = (next(res.results)).text
            speak += "The answer is " + answer + '.'
        elif "what is" in data or "who is" in data:
            app_id = "75H7YR-8T56GVTREH"
            client = wolframalpha.Client(app_id)
            ind = data.lower().split().index("is")
            data = data.split()[ind+1:]
            res = client.query(" ".join(data))
            answer = (next(res.results)).text
            speak += "The answer is " + answer + '.'
            
        response(speak)
        
    except sr.UnknownValueError:
        label.configure(text="Assistant could not understand the audio")
    except sr.RequestError as ex:
        label.configure(text="Request error from Google Speech Recognition" + ex)
    

def response(text):
    tts = gTTS(text=text, lang="en")
    audio = "Audio.mp3"
    tts.save(audio)
    playsound.playsound(audio)
    os.remove(audio)
    
engine = pyttsx3.init()
voices = engine.getProperty('voices')       
engine.setProperty('voice', voices[0].id)

window = Tk()
window.title("Voice Assistant")
window.configure(width=500, height=700)
window.configure(bg='lightgray')

label = Label(window, text='What can I help you with?')
label.pack(ipadx=30, ipady=18)
label.configure(bg='lightgray')

img = ImageTk.PhotoImage(Image.open("microphone_image.jpeg"))

button= Button(window, image=img,command=rec_audio,borderwidth=0)
button.pack(pady=30)

window.mainloop()